In [1]:
!pip install datasets

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import re
import os
import shutil
import datasets

from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures
# # from deep_translator import GoogleTranslator
from datasets import Dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import torch

# from sklearn.metrics import accuracy_score
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [3]:
from sklearn.model_selection import train_test_split

# Old data


In [4]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json"
f = open('laroseda_train.json')
# returns JSON object as
# a dictionary
train_data = json.load(f)

# Closing file
f.close()

--2024-10-27 22:20:49--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4231664 (4.0M) [text/plain]
Saving to: ‘laroseda_train.json.11’

laroseda_train.json 100%[===================>]   4.04M  --.-KB/s    in 0.01s   

2024-10-27 22:20:49 (348 MB/s) - ‘laroseda_train.json.11’ saved [4231664/4231664]



In [5]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json"
f = open('laroseda_test.json')
# returns JSON object as
# a dictionary
test_data = json.load(f)

# Closing file
f.close()


--2024-10-27 22:20:49--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025519 (1001K) [text/plain]
Saving to: ‘laroseda_test.json.11’

laroseda_test.json. 100%[===================>]   1001K  --.-KB/s    in 0.008s  

2024-10-27 22:20:49 (124 MB/s) - ‘laroseda_test.json.11’ saved [1025519/1025519]



In [6]:
def prep_doc(doc):
    cleaned = list()
    for i in doc:
        rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        rev = re.sub(r'\s+', ' ', rev)
        rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


def read_data(set):
    pos_data = list()
    neg_data = list()
    for i in set:
        # print(i)
        match i['starRating']:
            case '1':
                neg_data.append(i)
            case '2':
                neg_data.append(i)
            case '4':
                pos_data.append(i)
            case '5':
                pos_data.append(i)
    return neg_data, pos_data


def read_docs():
    f = open('laroseda_train.json')
    # returns JSON object as
    # a dictionary
    train_data = json.load(f)
    f.close()

    f = open('laroseda_test.json')
    # returns JSON object as
    # a dictionary
    test_data = json.load(f)
    # Closing file
    f.close()

    train_neg, train_pos = read_data(train_data['reviews'])
    test_neg, test_pos = read_data(test_data['reviews'])

    train_pos_clean = prep_doc(train_pos)
    train_neg_clean = prep_doc(train_neg)
    test_pos_clean = prep_doc(test_pos)
    test_neg_clean = prep_doc(test_neg)

    return train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean

In [7]:
# # Load and preprocess data
# train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

# train_texts = train_pos_clean + train_neg_clean
# train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)

# test_texts = test_pos_clean + test_neg_clean
# test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# New Data

In [9]:
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_1.json'
f = open('reviews_rating_1.json')
test_data_1 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_2.json'
f = open('reviews_rating_2.json')
test_data_2 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_3.json'
f = open('reviews_rating_3.json')
test_data_3 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_4.json'
f = open('reviews_rating_4.json')
test_data_4 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_5.json'
f = open('reviews_rating_5.json')
test_data_5 = json.load(f)
f.close()


--2024-10-27 22:20:50--  https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588174 (574K) [text/plain]
Saving to: ‘reviews_rating_1.json.11’

reviews_rating_1.js 100%[===================>] 574.39K  --.-KB/s    in 0.005s  

2024-10-27 22:20:50 (114 MB/s) - ‘reviews_rating_1.json.11’ saved [588174/588174]

--2024-10-27 22:20:50--  https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

In [10]:
def prep_doc_new(doc):
    cleaned = list()
    for rev in doc:
        if not isinstance(rev,str):
          continue
        # rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        rev = re.sub(r'\s+', ' ', rev)
        rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


In [11]:
# test_data_pos = test_data_1 + test_data_2 + test_data_4 + test_data_5
# data_clean_neg = prep_doc_new(test_data_1+test_data_2+test_data_3)
data_clean_neg = prep_doc_new(test_data_1+test_data_2)

data_clean_pos = prep_doc_new(test_data_4+test_data_5)

data_len=min(len(data_clean_pos),len(data_clean_neg))
data_clean_pos = data_clean_pos[:data_len]
data_clean_neg = data_clean_neg[:data_len]

# data_clean_pos = data_clean_pos[:10000]
# data_clean_neg = data_clean_neg[:10000]

y_opt = [1] * len(data_clean_pos) + [0] * len(data_clean_neg)
X_opt = data_clean_pos + data_clean_neg

print(len(data_clean_neg))
print(len(data_clean_pos))

train_texts, test_texts, train_labels, test_labels = train_test_split(X_opt, y_opt, test_size=0.2, random_state=42)

4257
4257


# Test

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]  # Unpack the first element of the tuple
    print("Logits shape:", logits.shape)
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1_score": f1}

def run_test(model_identifier, train_texts, test_texts, train_labels, test_labels):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_identifier)

    # Add a new padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = AutoModelForSequenceClassification.from_pretrained(model_identifier, num_labels=2)

    # Resize token embeddings after adding new special tokens
    model.resize_token_embeddings(len(tokenizer))

    # Set the padding token ID in the model configuration
    model.config.pad_token_id = tokenizer.pad_token_id

    # Tokenize the texts with padding and truncation
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=200)

    train_dataset = Dataset.from_dict({
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask'],
        'label': train_labels
    })

    test_dataset = Dataset.from_dict({
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask'],
        'label': test_labels
    })

    # Set the format of the datasets for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    training_args = TrainingArguments(
        output_dir="./results",
        save_strategy="no",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        logging_dir="./logs",
        report_to="none"  # Disable wandb

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Evaluation results: {eval_result}")

    return eval_result



In [21]:



names = [
         'dumitrescustefan/t5-v1_1-base-romanian'
        #  'racai/distilbert-base-romanian-uncased', #--
        #  'racai/distilbert-base-romanian-cased', #--
        #  'dumitrescustefan/bert-base-romanian-uncased-v1', #---
        #  'dumitrescustefan/bert-base-romanian-cased-v1', #---
        #  'xlm-roberta-base' #---
        #  'bert-base-multilingual-cased',  #---
        #  'bert-base-multilingual-uncased',  #---
        #  'gpt2',  # The standard GPT-2 model identifier #---
        #  'DGurgurov/xlm-r_romanian_sentiment' #---
        #  'readerbench/RoGPT2-base'

]

 #  'OpenLLM-Ro/RoLlama2-7b-Base',
 #  'OpenLLM-Ro/RoGemma-7b-Instruct',

# Run the test with GPT-2
eval_result = run_test(names[0], train_texts, test_texts, train_labels, test_labels)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of MT5ForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/t5-v1_1-base-romanian and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train encodings input_ids shape: torch.Size([6811, 200])
Train encodings attention_mask shape: torch.Size([6811, 200])
Test encodings input_ids shape: torch.Size([1703, 200])
Test encodings attention_mask shape: torch.Size([1703, 200])


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


IndexError: index -1 is out of bounds for dimension 1 with size 0

In [ ]:
breq

In [ ]:
torch.cuda.empty_cache()


In [ ]:
from google.colab import runtime
runtime.unassign()

In [26]:
# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # If the logits are in the form of a tuple, handle accordingly
    if isinstance(logits, tuple):
        logits = logits[0]  # Extract the first element if it's a tuple

    # Use only the first token's logits for classification
    logits = logits[:, 0, :]  # shape: (batch_size, num_labels)
    predictions = np.argmax(logits, axis=-1)

    # Ensure labels are flattened to match predictions shape
    labels = labels.flatten()

    # Calculate accuracy and F1 score
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')

    return {"accuracy": accuracy, "f1_score": f1}

# The rest of the code remains the same
# Function to run the classification task
def run_classification(train_texts, test_texts, train_labels, test_labels):
    # Load the tokenizer and model
    model_identifier = 'dumitrescustefan/t5-v1_1-base-romanian'
    tokenizer = AutoTokenizer.from_pretrained(model_identifier)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_identifier)

    # Check if the padding token is set
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # Resize token embeddings after adding new special tokens
    model.resize_token_embeddings(len(tokenizer))

    # Add a task prefix to texts
    task_prefix = "classify: "

    # Tokenize the training texts with padding and truncation
    train_encodings = tokenizer(
        [task_prefix + text for text in train_texts],
        truncation=True,
        padding='max_length',
        max_length=200,
        return_tensors='pt'
    )

    # Tokenize the test texts
    test_encodings = tokenizer(
        [task_prefix + text for text in test_texts],
        truncation=True,
        padding='max_length',
        max_length=200,
        return_tensors='pt'
    )

    # Convert labels to text format and tokenize
    train_labels_text = [str(label) for label in train_labels]  # Convert labels to strings
    test_labels_text = [str(label) for label in test_labels]  # Convert labels to strings

    # Tokenize the labels as inputs for the model
    train_labels_encodings = tokenizer(
        train_labels_text,
        truncation=True,
        padding='max_length',
        max_length=1,  # Each label is a single token
        return_tensors='pt'
    )

    test_labels_encodings = tokenizer(
        test_labels_text,
        truncation=True,
        padding='max_length',
        max_length=1,  # Each label is a single token
        return_tensors='pt'
    )

    # Create datasets
    train_dataset = Dataset.from_dict({
        'input_ids': train_encodings['input_ids'].numpy(),
        'attention_mask': train_encodings['attention_mask'].numpy(),
        'labels': train_labels_encodings['input_ids'].numpy()  # Use encoded labels
    })

    test_dataset = Dataset.from_dict({
        'input_ids': test_encodings['input_ids'].numpy(),
        'attention_mask': test_encodings['attention_mask'].numpy(),
        'labels': test_labels_encodings['input_ids'].numpy()  # Use encoded labels
    })

    # Set the format of the datasets for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        save_strategy="no",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=32,
        eval_strategy="epoch",  # Update to the new argument
        logging_dir="./logs",
        report_to="none"  # Disable wandb
    )

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Evaluation results: {eval_result}")

    return eval_result

# Run the classification task
eval_result = run_classification(train_texts, test_texts, train_labels, test_labels)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,2.962600,0.000000,1.000000,1.000000
2,0.047300,0.000000,1.000000,1.000000
3,0.003500,0.000000,1.000000,1.000000


Evaluation results: {'eval_loss': 0.0, 'eval_accuracy': 1.0, 'eval_f1_score': 1.0, 'eval_runtime': 16.7223, 'eval_samples_per_second': 101.84, 'eval_steps_per_second': 3.229, 'epoch': 3.0}


In [28]:
print(train_labels)
print(test_labels)

[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 

In [27]:
# Example to check for overlap
print("Unique training labels:", set(train_labels))
print("Unique testing labels:", set(test_labels))


Unique training labels: {0, 1}
Unique testing labels: {0, 1}


In [29]:
print("Sample training texts and labels:")
for i in range(5):
    print(train_texts[i], train_labels[i])

print("Sample testing texts and labels:")
for i in range(5):
    print(test_texts[i], test_labels[i])


Sample training texts and labels:
deseori se schimba pozitia produselor , ceea ce ma descurajeaza sa caut un anumit produs , cand nu am prea mult timp la dispozitie 1
termosul este foarte frumos , placut la atingere , capacele se inchid usor ( daca ai timp sa alegi unul care se inchide usor ) insa capacul interior nu se inchide perfect si scapa lichid . oricat de tare l-am strans , daca il agit in mana tot scapa lichid . nu recomand sistemul ! ! ! mai bun e cel clasic cu buton .  0
daca se poate sa se promoveze si alte brand-uri de produse cu preturi medii pentru o gama mai variata de produse si libertate de alegere eventual chiar si produse romanesti / produse fabricate in romania .  1
nu am nimic de comentat , dar as dori sa aveti si marimi f mari 1
materialele din care sunt confectionate nu respira , ca urmare se imbiba de transpiratie si devin urat mirositoare dupa un interval nu prea lung de folosire . de asemenea cusaturile cedeaza destul de repede . . .  0
Sample testing texts a